## MTA Ridership 2023
Spotting peak destination and origin stations, and potential clusters of stations with high ridership.
Database has been 'cleaned' beforehand, before exporting from data.ny.gov, reducing file size and removing repetitive columns with similar info but differnt format.

test with query by api / cloud sql for year data - expand scope, then join population density with station density, normalised as crowd(divided-by)/station index (the closer to 1 the better), based on min and max values - then derive districts that are underserved/ overserved (using conditions 0.1 - 0.5 - 0.8) underserved - optimally served - overserved 

ref: https://github.com/mebauer/mta-data/blob/main/data-exporter.py

In [ ]:
%pip install duckdb

import pandas as pd
import duckdb
from datetime import datetime
import time

export_log = "export-log.txt"
api_base = "https://data.ny.gov/resource/uhf3-t34z.csv"
db_file = "subway-trips.db"

limit = 1_000_000
offset = 0

# Function to log messages
def log_message(message: str) -> None:
    """Log messages with timestamp."""
    with open(export_log, 'a') as file:
        file.write(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')},{message}\n")

with duckdb.connect(db_file) as con:
    
    while True:
        
        api_path = f"{api_base}?$limit={limit}&$offset={offset}&$order=:id"
        log_message(f"Fetching data from: {api_path}")
        
        try:
            df = pd.read_csv(api_path)
            rows = df.shape[0]
            
            if offset == 0:
                # Create the table if it's the first chunk
                con.execute("CREATE TABLE trips AS SELECT * FROM df")
            else:
                # Insert data into the existing table
                con.execute("INSERT INTO trips SELECT * FROM df")

            if rows < limit:
                # If fewer rows than limit, we are done
                log_message("Finished processing all data.")
                break

            # Update offset for the next batch
            offset += limit

            # Sleep to avoid hitting rate limits
            time.sleep(10)
            
        except Exception as e:
            log_message(f"Error reading data: {e}")
            break


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ----------- ---------------------------- 3.1/11.0 MB 15.3 MB/s eta 0:00:01
   ---------------------- ----------------- 6.0/11.0 MB 15.4 MB/s eta 0:00:01
   ---------------------------------- ----- 9.4/11.0 MB 15.4 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 14.9 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))